# Housing Price Prediction

### Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

In [ ]:
data = pd.read_csv("Data/bengaluru_house_prices.csv")

In [ ]:
data.shape

In [ ]:
data = data.drop(["area_type", "society", "balcony", "availability"], axis="columns")
data

### Data Cleaning

In [ ]:
data.isnull().sum()

In [ ]:
data = data.dropna()
data.isna().sum()

In [ ]:
data['size'].unique()

In [ ]:
data = data.copy()
data.loc[:, "BHK"] = data['size'].apply(lambda x: int(x.split(' ')[0]))

In [ ]:
data["BHK"].unique()

In [ ]:
data.total_sqft.unique()

In [ ]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [ ]:
data[~data["total_sqft"].apply(is_float)]

In [ ]:
def convert_sqft_to_num(x):
    ranges = x.split('-')
    if len(ranges) == 2:
        return (float(ranges[0]) + float(ranges[1]))/2
    try:
        return float(x)
    except:
        return None

In [ ]:
data = data.copy()
data['total_sqft'] = data['total_sqft'].apply(convert_sqft_to_num)

In [ ]:
data

### Feature Engineering

In [ ]:
data = data.copy()

In [ ]:
data['price_per_sqft'] = (data['price'] * 100000) / data['total_sqft']
data

In [ ]:
len(data['location'].unique())

In [ ]:
data.location = data.location.apply(lambda x: x.strip())
location_statistics = data.groupby('location')['location'].agg('count').sort_values(ascending=False)
location_statistics

In [ ]:
len(location_statistics[location_statistics <= 10])

In [ ]:
location_statistics_less_than_10 = location_statistics[location_statistics <= 10]

In [ ]:
data.location = data.location.apply(lambda x: 'Other' if x in location_statistics_less_than_10 else x)

In [ ]:
data

### Outlier Removal

In [ ]:
data[data.total_sqft/data.BHK < 300]

In [ ]:
data = data[~(data.total_sqft/data.BHK < 300)]

In [ ]:
data

In [ ]:
data.price_per_sqft.describe()

In [ ]:
def remove_outliers(dataFrame):
    data_frame_out = pd.DataFrame()
    for key, sub_data_frame in dataFrame.groupby('location'):
        mean = np.mean(sub_data_frame.price_per_sqft)
        std = np.std(sub_data_frame.price_per_sqft)
        reduced_data_frame = sub_data_frame[(sub_data_frame.price_per_sqft > (mean - std)) & 
                                            (sub_data_frame.price_per_sqft <= (mean + std))]
        data_frame_out = pd.concat([data_frame_out, reduced_data_frame], ignore_index=True)
    return data_frame_out

In [ ]:
data = remove_outliers(data)

In [ ]:
data

In [ ]:
def plot_scatter_graph(data, location):
    bhk_2 = data[(data.location == location) & (data.BHK == 2)]
    bhk_3 = data[(data.location == location) & (data.BHK == 3)]
    plt.rcParams['figure.figsize'] = (10, 6)
    plt.scatter(bhk_2.total_sqft, bhk_2.price, color='blue', label='2 BHK', s=50)
    plt.scatter(bhk_3.total_sqft, bhk_3.price, marker='+', color='green', label='3 BHK', s=50)
    plt.xlabel("Total Sqaure Feet Area")
    plt.ylabel("Price")
    plt.title(location)
    plt.legend()

In [ ]:
plot_scatter_graph(data, "Yeshwanthpur")

In [ ]:
def remove_bhk_outliers(data):
    indices = np.array([])
    for location, location_data_frame in data.groupby('location'):
        bhk_statistics = {}
        for bhk, bhk_data_frame in location_data_frame.groupby('BHK'):
            bhk_statistics[bhk] = {
                'mean': np.mean(bhk_data_frame.price_per_sqft),
                'std': np.std(bhk_data_frame.price_per_sqft),
                'count':bhk_data_frame.shape[0]
            }
        for bhk, bhk_data_frame in location_data_frame.groupby('BHK'):
            statistics = bhk_statistics.get(bhk - 1)
            if statistics and statistics['count'] > 5:
                indices = np.append(indices, bhk_data_frame[bhk_data_frame.price_per_sqft < (statistics['mean'])].index.values)
    return data.drop(indices, axis='index')

In [ ]:
data = remove_bhk_outliers(data)

In [ ]:
data

In [ ]:
plot_scatter_graph(data, "Yeshwanthpur")

In [ ]:
plt.rcParams['figure.figsize'] = (10, 6)
plt.hist(data.price_per_sqft, rwidth=0.75)
plt.xlabel('Price Per Square Feet')
plt.ylabel('Count')
plt.show()

In [ ]:
data.bath.unique()

In [ ]:
plt.rcParams['figure.figsize'] = (10, 6)
plt.hist(data.bath, rwidth=0.75)
plt.xlabel('Price Per Square Feet')
plt.ylabel('Count')
plt.show()

In [ ]:
data = data[data.bath < data.BHK + 2]
data

### Model Building

In [ ]:
data = data.drop(['size', 'price_per_sqft'], axis='columns')

In [ ]:
data

In [ ]:
dummies = pd.get_dummies(data.location)

In [ ]:
data = pd.concat([data, dummies.drop('Other', axis='columns')], axis='columns')

In [ ]:
data = data.drop('location', axis='columns')

In [ ]:
data

In [ ]:
X = data.drop('price', axis='columns')
y = data.price

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [ ]:
model_linear_regression = LinearRegression()

In [ ]:
model_linear_regression.fit(X_train, y_train)

In [ ]:
model_linear_regression.score(X_test, y_test)

In [ ]:
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

In [ ]:
cross_val_score(LinearRegression(), X, y, cv=cv)

In [ ]:
def find_best_model(X, y):
    algorithms = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params' : {}
        }, 
        'lasso':{
            'model':Lasso(),
            'params': {}
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {}
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algorithm_name, configuration in algorithms.items():
        grid_search = GridSearchCV(configuration['model'], configuration['params'], cv=cv, return_train_score=False)
        grid_search.fit(X, y)
        scores.append({
            'model': algorithm_name,
            'best_score': grid_search.best_score_,
            
        })
    return pd.DataFrame(scores, columns=['model', 'best_score'])

In [ ]:
find_best_model(X, y)